# 실습 세션 2: AutoGen 기반 여행 플래너 Multi-Agent 앱 구현

이번 세션에서는 GitHub Copilot의 Agent Mode와 Microsoft AutoGen 프레임워크를 활용하여 여행 플래너 멀티 에이전트 애플리케이션을 구현합니다. 다양한 역할의 AI 에이전트가 협업하여 사용자의 여행 계획을 도와주는 시스템을 만들어봅니다.

## 목차

1. GitHub Copilot 활성화 및 Agent Mode 소개

2. MCP 코드 생성 및 실행 (참고용)

3. AutoGen 프레임워크란?

4. AutoGen 프레임워크 및 환경 설정

5. 여행 플래너 Multi-Agent 시스템 구성

6. 그룹 채팅 및 종료 조건 설정

7. 여행 플래너 Multi-Agent 대화 실행

8. AutoGen Studio 실행 가이드

9. 마무리 및 다음 단계

---

각 섹션에서는 Copilot/AutoGen 개념, 환경 설정, 멀티 에이전트 구조, 실전 예제, 그리고 직접 실습할 수 있는 셀을 제공합니다.

## 1. GitHub Copilot 활성화 및 Agent Mode 소개

이 실습에서는 GitHub Copilot의 Agent Mode와 함께, Vibe Coding 환경을 활용합니다. 아래 내용을 참고하여 Copilot을 활성화하고, Edit Mode/Chat Mode의 차이와 사용법, 그리고 Vibe Coding의 장점을 익히세요.

### GitHub Copilot 활성화 방법

#### VS Code 확장 프로그램 설치
- VS Code 좌측 사이드바에서 Extensions(확장) 아이콘을 클릭합니다.
- 검색창에 `GitHub Copilot`을 입력하고, 공식 확장 프로그램을 설치합니다.

#### GitHub 계정 로그인 및 Copilot 활성화
- 설치 후, VS Code 우측 하단 또는 명령 팔레트(Cmd+Shift+P)에서 `GitHub: Sign in`을 선택해 GitHub 계정으로 로그인합니다.
- 로그인 후, Copilot이 정상적으로 활성화되면 에디터에서 자동완성, Chat, Agent Mode 기능을 사용할 수 있습니다.

#### Agent Mode 사용
- 에디터 우측 하단의 Copilot 아이콘을 클릭하거나, 명령 팔레트에서 `Copilot: Chat` 또는 `Copilot: Enable Agent Mode`를 실행해 다양한 모드를 체험할 수 있습니다.
- 파일 편집(Edit Mode), 대화(Chat Mode) 등 다양한 개발 작업을 Copilot이 지원합니다.

> 참고: Copilot은 유료 플랜이 필요할 수 있으며, 조직 정책에 따라 사용이 제한될 수 있습니다. 자세한 내용은 [GitHub Copilot 공식 문서](https://docs.github.com/en/copilot)에서 확인하세요.

### Vibe Coding이란?
- **Vibe Coding**은 Andrej Karpathy가 2025년 제안한 새로운 코딩 방식으로, "코드의 존재 자체를 잊고, 원하는 것을 자연어로 말하면 AI가 코드를 만들어주는" 대화형·실험적 프로그래밍 패러다임입니다.
- 사용자는 문제나 요구사항을 자연어(프롬프트)로 LLM(대형 언어 모델)에 설명하고, LLM이 코드를 생성합니다. 이때 사용자는 코드를 직접 작성하거나 모든 내용을 이해하지 않아도, AI가 만들어주는 결과를 실험·검증하며 반복적으로 개선합니다.
- Karpathy는 "그냥 보고, 말하고, 실행하고, 복사-붙여넣기 하면 대부분 동작한다"고 표현했으며, 이 과정은 음성이나 채팅 등 대화형 인터페이스로 진행될 수 있습니다.
- Vibe Coding은 빠른 프로토타이핑, 개인 프로젝트, 실험적 개발에 적합하며, 코드의 완전한 이해 없이도 결과를 얻을 수 있다는 점이 특징입니다. 하지만, 복잡한 서비스나 프로덕션 환경에서는 코드 품질·보안·유지보수 측면에서 한계가 있으므로 주의가 필요합니다.

> 참고: Vibe Coding은 특정 플랫폼이나 협업 환경이 아니라, AI와 자연어 프롬프트를 활용한 실험적·대화형 코드 생성 패러다임입니다. (출처: [Karpathy X 원문](https://x.com/karpathy/status/1886192184808149383?lang=en))

### Agent Mode란?
- **Agent Mode**는 Copilot이 단순 코드 자동완성(Inline) 기능을 넘어, 파일 편집(Edit Mode)과 대화(Chat Mode) 기반의 다양한 개발 작업을 지원하는 모드입니다.
- Edit Mode/Chat Mode 모두 Agent Mode의 일부로, 더 복잡한 작업을 Copilot이 직접 수행할 수 있습니다.
- 자연어로 시스템 요구사항, 코드 생성, 리팩터링, 테스트, 문서화 등 다양한 개발 작업을 자동화할 수 있습니다.

### Agent Mode 활용 예시

아래는 Agent Mode에서 실제로 많이 활용되는 대표적인 5가지 시나리오입니다. 각 유형별로 입력 프롬프트 예시와 함께 설명합니다.

#### 시스템/서비스 구조 자동 생성
- 자연어로 시스템 요구사항, 역할, 예시 프롬프트 등을 입력하면, Copilot이 전체 구조와 주요 역할별 프롬프트, 파일 구조, 클래스/함수 시그니처 등을 자동으로 제안합니다.
- **입력 프롬프트 예시:**
    ```
    AutoGen 기반 여행 플래너 시스템의 전체 구조와 주요 역할별 프롬프트 예시를 만들어줘.
    ```

#### 프로토콜/스펙 기반 서버·클라이언트 코드 자동 생성
- 특정 프로토콜(MCP, A2A 등)이나 API 명세, 데이터 포맷을 입력하면, Copilot이 관련 Python/TypeScript 등 서버·클라이언트 코드를 자동으로 생성합니다.
- **입력 프롬프트 예시:**
    ```
    MCP(Model Context Protocol) 프로토콜에 맞는 Python 서버와 클라이언트 코드를 생성해줘.
    ```

#### 테스트 코드, 시나리오, 외부 문서 자동 생성 및 확장
- 생성된 코드에 대해 테스트 코드, 추가 시나리오, README 등 코드 외부 산출물을 자연어로 요청하면 Copilot이 자동으로 생성/확장합니다.
- **입력 프롬프트 예시:**
    ```
    위 MCP 서버 코드에 대한 pytest 기반 테스트 코드를 만들어줘.
    README 파일을 생성해줘.
    ```

#### 기존 코드 리팩터링 및 버그 수정
- 기존 코드의 리팩터링, 성능 개선, 버그 수정 등도 자연어로 요청하면 Copilot이 자동으로 제안합니다.
- **입력 프롬프트 예시:**
    ```
    아래 Python 함수의 성능을 개선하고, 불필요한 반복문을 제거해줘.
    ```

#### 코드 내부 문서화 및 주석 자동 생성
- 함수/클래스의 docstring, 주석 등 코드 내부 문서화를 자연어로 요청하면 Copilot이 자동으로 생성합니다.
- **입력 프롬프트 예시:**
    ```
    아래 Python 클래스에 Google Style docstring을 추가해줘.
    이 함수에 상세 주석을 달아줘.
    ```

---
Agent Mode를 활용하면 복잡한 시스템 구조, 서버-클라이언트 통신, 테스트 코드, 문서 등 다양한 개발 작업을 단계별로 자동 생성하고 확장할 수 있습니다. 실제 서비스 수준의 코드와 문서를 빠르게 확보할 수 있으니, 다양한 시나리오를 실습하며 경험을 쌓아보세요.

> ⚠️ 참고: 복잡한 시스템이나 실제 서비스 수준의 코드를 만들 때는 여러 번의 반복적인 프롬프트 입력과 Copilot의 제안 결과를 꼼꼼히 확인·수정하는 과정이 필요합니다. 한 번의 프롬프트로 완성되는 경우는 드물며, 요구사항/명세/예시를 구체적으로 입력하고, 결과를 검토·수정·재요청하는 반복이 중요합니다.

### GitHub Copilot이 지원하는 LLM 모델 (2025년 5월 기준)

아래는 VS Code에서 Copilot이 제공하는 대표적인 LLM 모델 목록과 간단한 설명입니다. 실제 사용 가능 여부는 계정, 조직 정책, 지역, 서비스 업데이트에 따라 달라질 수 있습니다.


- **Claude 3.5 Sonnet / Claude 3.7 Sonnet**: Anthropic의 고성능 LLM. 대규모 문서 요약, 분석, 자연어 처리에 강점.
- **Gemini 2.5 Pro (Preview)**: Google의 최신 LLM. 코드 생성, 자연어 처리, 멀티모달 작업 지원.
- **GPT-4.1**: OpenAI의 최신 고성능 LLM. 코드, 문서, 자연어 작업에 모두 강력하며 Copilot에서 기본적으로 제공됨.
- **GPT-4o**: OpenAI의 멀티모달(텍스트·음성·이미지) 지원 LLM. 빠른 응답과 다양한 입력 지원.
- **o4-mini (Preview)**: 경량화된 OpenAI LLM으로, 빠른 코드 생성과 실시간 응답에 적합. 일부 환경에서 프리뷰로 제공.


> Copilot 모델 선택은 VS Code 우측 하단 Copilot 패널에서 직접 변경할 수 있습니다. 최신 정보는 [GitHub Copilot 공식 문서](https://docs.github.com/en/copilot/using-github-copilot/ai-models/choosing-the-right-ai-model-for-your-task) 및 각 LLM 제공사의 공식 문서를 참고하세요.

## 2. MCP 코드 생성 및 실행 (참고용)

> ⚠️ 이 섹션의 예시는 실습이 아닌 참고용 가이드입니다. 실제 환경에서는 요구사항과 명세를 구체적으로 입력하고, Copilot의 제안 결과를 반복적으로 검토·수정하세요.

참고 자료
- [MCP 공식 스펙](https://modelcontextprotocol.io/specification/2025-03-26)
- [MCP 서버 퀵스타트](https://modelcontextprotocol.io/quickstart/server)

---

### MCP 서버 코드 예시

아래는 최신 MCP(2025-03-26) 스펙 기반의 FastMCP Python 서버 예시입니다.

```python
    from mcp.server.fastmcp import FastMCP

    mcp = FastMCP("example-server")

    @mcp.tool()
    def echo(message: str) -> str:
        """입력받은 메시지를 그대로 반환합니다."""
        return message

    @mcp.tool()
    def info() -> str:
        """서버의 간단한 정보를 반환합니다."""
        return "MCP Example Server (2025-03-26 spec)"

    if __name__ == "__main__":
        mcp.run(transport="stdio")
```

---

### MCP 서버 실행 및 VS Code 연동

1. MCP 서버 실행
    ```python
    uv run <서버파일명>.py
    ```
2. Copilot(Agent Mode) 확장 설치 및 MCP Host 기능 활성화
3. MCP 서버 등록 예시 (`~/.copilot/mcp_servers.json`)
    ```json
    {
      "mcpServers": {
        "example-server": {
          "command": "uv",
          "args": [
            "--directory",
            "/ABSOLUTE/PATH/TO/SERVER/FOLDER",
            "run",
            "<서버파일명>.py"
          ]
        }
      }
    }
    ```
4. VS Code 재시작 또는 Copilot 확장 reload
5. Copilot 패널에서 MCP 서버 인식 확인

---

### Copilot Chat에서 MCP 툴 호출

- 명령어 직접 호출:
    ```python
    /agent tool:echo message="Hello, MCP!"
    /agent tool:info
    ```
- 자연어 프롬프트 예시:
    ```
    메시지 '테스트'를 echo 툴로 반환해줘.
    서버의 정보를 알려줘.
    ```

> MCP 서버가 정상적으로 연동되어 있어야 하며, 툴 목록은 Copilot 패널에서 확인할 수 있습니다.

---

이제 Agent Mode에서 MCP 서버 코드 작성, 클라이언트 연동, Copilot Chat에서의 호출까지 한 번에 실습할 수 있습니다.

## 3. AutoGen 프레임워크란?

AutoGen은 Microsoft에서 개발한 오픈소스 프레임워크로, 다양한 역할의 AI 에이전트(Assistant, UserProxy 등)를 손쉽게 정의하고, 이들이 협업하는 Multi-Agent 시스템을 빠르게 구축할 수 있도록 지원합니다.

### AutoGen Multi-Agent 아키텍처 다이어그램
```
[User/Task]
   |
   v
[UserProxyAgent]
   |
   v
+-----------------------------+
|         GroupChat           |
|  (RoundRobin/Selector 등)   |
+-----------------------------+
   |        |         |      |
   v        v         v      v
[AssistantAgent1] [AssistantAgent2] ... [AssistantAgentN]
   |        |         |      |
   v        v         v      v
[ModelClient/LLM API]
   |
   v
[응답/결과]
```
- 일반적으로 사용자의 입력은 UserProxyAgent를 통해 시작되며, GroupChat 내에서 여러 AssistantAgent와 상호작용이 이루어집니다.
- GroupChat은 UserProxyAgent와 여러 AssistantAgent를 멤버로 포함하며, 메시지 전달 방식(라운드로빈, 셀렉터 등)에 따라 대화가 진행됩니다.
- 각 AssistantAgent는 자신의 역할에 따라 LLM API(ModelClient)를 호출해 응답을 생성합니다.
- 종료 조건(TerminationCondition)에 따라 대화가 종료되고, 최종 결과가 반환됩니다.

### 주요 특징
- 다양한 LLM(OpenAI, Azure OpenAI 등)과 연동 가능
- 에이전트별 역할, 프롬프트, 종료 조건 등 유연하게 설정
- 팀 기반 협업, 그룹 채팅, 도구 연동 등 고급 시나리오 지원
- 실시간 스트리밍, 대화 기록, UI 연동 등 실전 활용에 최적화

### 주요 컴포넌트
- **AssistantAgent**: 특정 역할(예: 여행 플래너, 로컬 추천 등)을 수행하는 AI 에이전트입니다. 각 에이전트는 고유의 시스템 메시지와 프롬프트, 모델 클라이언트를 가질 수 있습니다.
- **UserProxyAgent**: 실제 사용자를 대신해 요청을 전달하거나, 에이전트와 상호작용하는 프록시 역할의 에이전트입니다. GroupChat에 포함되어 사용자 입력을 시작점으로 메시지 흐름에 참여합니다.
- **GroupChat / RoundRobinGroupChat**: 여러 에이전트가 팀을 이루어 협업하는 구조를 정의합니다. 메시지 전달 방식(라운드로빈, 셀렉터 등)을 설정할 수 있습니다.
- **TerminationCondition**: 대화 종료 조건을 정의합니다. (예: 최대 메시지 수, 특정 키워드 도달 등)
- **ModelClient**: OpenAI, Azure OpenAI 등 다양한 LLM API와 연동하는 클라이언트입니다.
- **Console / UI**: 대화 과정을 콘솔 또는 웹 UI로 시각화할 수 있습니다.

AutoGen을 활용하면 복잡한 AI 에이전트 협업 시나리오를 코드 몇 줄로 손쉽게 구현할 수 있습니다.

---

## 4. AutoGen 프레임워크 및 환경 설정

이 단계에서는 AutoGen 프레임워크를 실습 환경에 설치하고, Azure OpenAI 등 LLM 서비스와 연동할 수 있도록 환경 변수를 설정합니다.

- requirements.txt를 활용해 필요한 라이브러리와 AutoGen 패키지를 설치합니다.
- .env 파일에 입력한 환경 변수 정보를 불러와 Azure OpenAI 등 LLM API와 연동할 수 있도록 모델 클라이언트를 준비합니다.
- 환경 설정이 올바르게 완료되면, 이후 다양한 에이전트와 팀 구조를 자유롭게 실습할 수 있습니다.
- 만약 연결 오류가 발생하면, 환경 변수 값과 패키지 설치 상태를 다시 확인하세요.

### 패키지 설치 및 커널 재시작 안내

- 아래 셀을 실행하여 필요한 라이브러리와 AutoGen 패키지를 설치하세요.
- 설치가 완료된 후에는 **커널을 반드시 재시작**해야 모든 패키지가 정상적으로 인식됩니다.
- 커널 재시작 방법: 메뉴에서 'Runtime' > 'Restart Kernel' 또는 상단의 재시작 아이콘 클릭

In [1]:
# AutoGen 및 필요한 라이브러리 설치
%pip install -r requirements.txt


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 환경 변수 설정 및 모델 클라이언트 준비
OpenAI API 키 등 환경 변수를 설정하고, 모델 클라이언트를 준비합니다.

In [1]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

import os
import json
from dotenv import load_dotenv

load_dotenv()

model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), # For key-based authentication.
)

print("AutoGen 설정이 완료되었습니다.")

AutoGen 설정이 완료되었습니다.


## 5. 여행 플래너 Multi-Agent 시스템 구성

이 단계에서는 다양한 역할의 에이전트(여행 플래너, 로컬 추천, 언어 팁, 여행 요약 등)를 정의하여 실제 Multi-Agent 구조를 설계합니다.

- 각 에이전트는 고유의 역할, 시스템 메시지, 프롬프트, 모델 클라이언트를 가집니다.
- 실습을 통해 여러 에이전트가 협업하여 복잡한 문제(여행 일정 추천 등)를 해결하는 구조를 직접 구현해볼 수 있습니다.
- 에이전트별 역할 분담, 상호작용 방식, 종료 조건 등을 자유롭게 실험하며, 실제 서비스에 적용 가능한 멀티 에이전트 설계 역량을 키울 수 있습니다.
- 아래 예시 코드를 참고하여 자신만의 에이전트 구조를 추가하거나 수정해보세요.

In [2]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent

planner_agent = AssistantAgent(
    name="planner_agent",
    description="여행 일정을 계획해주는 에이전트입니다.",
    model_client=model_client,
    system_message="""당신은 사용자의 요청에 따라 여행 일정을 제안하는 유용한 여행 플래너 에이전트입니다. 여행 계획을 마무리할 때는 반드시 'INITIAL PLAN COMPLETE'로 끝내세요."""
)

local_agent = AssistantAgent(
    name="local_agent",
    description="현지의 숨은 명소와 로컬 경험을 추천하는 에이전트입니다.",
    model_client=model_client,
    system_message="""당신은 사용자를 위해 현지의 진짜 명소, 체험, 음식 등 흥미로운 로컬 활동을 추천하는 에이전트입니다. 추천을 마칠 때는 반드시 'LOCAL RECOMMENDATIONS COMPLETE'로 끝내세요."""
)

language_agent = AssistantAgent(
    name="language_agent",
    description="여행지에서의 언어/소통 팁을 제공하는 에이전트입니다.",
    model_client=model_client,
    system_message="""당신은 여행 계획을 검토하고, 해당 여행지에서 언어 또는 소통에 있어 중요한 팁과 주의사항을 제공하는 에이전트입니다. 언어 팁을 마칠 때는 반드시 'LANGUAGE TIPS COMPLETE'로 끝내세요."""
)

travel_summary_agent = AssistantAgent(
    name="travel_summary_agent",
    description="모든 에이전트의 제안을 종합하여 최종 여행 일정을 요약하는 에이전트입니다.",
    model_client=model_client,
    system_message="""당신은 다른 에이전트들의 제안과 조언을 모두 반영하여 최종 여행 일정을 상세하게 요약하는 에이전트입니다. 반드시 모든 관점을 통합한 최종 일정을 제공해야 하며, 계획이 완성되면 'TERMINATE'로 응답을 마무리하세요."""
)

user_proxy_travel = UserProxyAgent(
    name="user_proxy",
    description="여행 계획을 요청하는 사용자 역할의 에이전트입니다."
)

## 6. 그룹 채팅 및 종료 조건 설정

이 단계에서는 여러 에이전트가 협업하는 그룹 채팅 구조를 만들고, 대화 종료 조건을 설정합니다.

- AutoGen의 그룹 채팅 기능을 활용해 에이전트 간 메시지 전달 방식을 정의합니다.
- 대표적으로 **RoundRobinGroupChat**(에이전트가 순서대로 메시지를 주고받음), **SelectorGroupChat**(상황에 따라 특정 에이전트만 선택적으로 메시지를 주고받음) 등의 옵션이 있습니다.
- MaxMessageTermination, TextMentionTermination 등 다양한 종료 조건을 설정하여 대화 흐름을 제어할 수 있습니다.
- 실습을 통해 실제 서비스에서 필요한 협업 구조와 종료 조건을 자유롭게 실험해볼 수 있습니다.
- 아래 예시 코드를 참고하여 자신만의 그룹 구조와 종료 조건을 추가하거나 수정해보세요.

### 주요 그룹 채팅 옵션
- **RoundRobinGroupChat**: 모든 에이전트가 정해진 순서대로 차례로 메시지를 주고받는 구조입니다. 협업의 흐름이 예측 가능하고, 각 에이전트가 반드시 참여해야 하는 시나리오에 적합합니다.
- **SelectorGroupChat**: 매 라운드마다 상황에 따라 특정 에이전트만 선택적으로 메시지를 주고받는 구조입니다. 동적 역할 분배, 조건부 참여 등 유연한 협업 시나리오에 적합합니다.
- (AutoGen 버전에 따라) **CustomGroupChat** 등 직접 메시지 전달 로직을 구현해 확장할 수도 있습니다.

### 주요 종료 조건 예시
- **MaxMessageTermination**: 대화에서 주고받는 메시지의 최대 개수를 지정합니다. 예를 들어, `MaxMessageTermination(max_messages=30)`은 30개의 메시지가 오가면 대화를 자동으로 종료합니다. 무한 루프 방지, 리소스 절약 등에 유용합니다.
- **TextMentionTermination**: 특정 키워드가 메시지에 등장하면 대화를 종료합니다. 예를 들어, `TextMentionTermination("TERMINATE")`는 에이전트가 'TERMINATE'라는 단어를 응답에 포함하면 대화를 종료합니다. 명시적 종료 신호가 필요한 시나리오에 적합합니다.
- **Custom Termination**: 필요에 따라 직접 종료 조건을 구현할 수도 있습니다. 예를 들어, 특정 에이전트의 응답 패턴, 시간 제한, 외부 이벤트 등 다양한 조건을 활용할 수 있습니다.

이러한 옵션을 조합해 실제 서비스 요구에 맞는 협업 구조와 종료 조건을 설계해보세요.

In [3]:
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat

termination = MaxMessageTermination(max_messages=30) | TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent, user_proxy_travel],
    termination_condition=termination
)

## 7. 여행 플래너 Multi-Agent 대화 실행

이 단계에서는 앞서 정의한 다양한 에이전트와 그룹 채팅, 종료 조건을 실제로 실행하여 여행 계획을 완성하는 과정을 실습합니다.

- 사용자의 여행 요청(task)에 대해 여러 에이전트가 협업하여 답변을 생성합니다.
- 각 에이전트는 자신의 역할에 따라 정보를 제공하고, 그룹 채팅 구조와 종료 조건에 따라 대화가 진행 및 종료됩니다.
- 실습을 통해 멀티 에이전트 시스템의 실제 동작 방식과 대화 흐름을 직접 경험할 수 있습니다.
- 아래 예시 코드를 수정하여 다양한 여행 시나리오, 에이전트 구성, 종료 조건 등을 실험해보세요.

In [4]:
from autogen_agentchat.ui import Console

travel_task = """저는 친구와 함께 5일간 파리로 여행을 계획하고 있습니다. 저희는 예술과 음식을 좋아하며, 유명한 명소뿐만 아니라 현지의 숨은 보석 같은 장소도 경험하고 싶습니다. 저희는 프랑스어를 할 줄 모릅니다. 상세한 여행 일정을 만들어주세요."""

await Console(group_chat.run_stream(task=travel_task))

await model_client.close()

---------- TextMessage (user) ----------
저는 친구와 함께 5일간 파리로 여행을 계획하고 있습니다. 저희는 예술과 음식을 좋아하며, 유명한 명소뿐만 아니라 현지의 숨은 보석 같은 장소도 경험하고 싶습니다. 저희는 프랑스어를 할 줄 모릅니다. 상세한 여행 일정을 만들어주세요.
---------- TextMessage (planner_agent) ----------
파리에서 예술과 음식을 좋아하는 두 분을 위한 5일 여행 일정을 상세히 제안해 드리겠습니다. 프랑스어를 모르셔도 편안하게 즐길 수 있도록 영어 안내가 잘 되는 장소 위주로 구성하였고, 유명 명소뿐 아니라 현지인들이 사랑하는 숨은 보석 같은 장소도 포함했습니다.

---
### 1일차: 파리에 도착 & 마레 지구 탐방
- 오전: 파리 도착 후 호텔 체크인 및 휴식
- 오후: 마레 지구(Le Marais) 산책  
  - **피카소 미술관** (Musée Picasso): 피카소 작품 감상 (영어 오디오 가이드 대여 가능)  
  - **카페 그레고아르**(Café Charlot)에서 가벼운 프랑스식 브런치 즐기기  
  - 마레 지구의 부티크 샵, 아트 갤러리 탐방  
- 저녁: 마레 지구의 숨은 보석 레스토랑 **Les Philosophes**에서 프랑스 가정식 만끽 (영어 메뉴 지원)

---
### 2일차: 미술관과 센 강 산책
- 오전: **루브르 박물관** 방문 (입구에서 영어 지도와 안내 받기 추천)  
  - 모나리자, 밀로의 비너스 등 명작 감상  
- 점심: 루브르 근처 **Le Fumoir** 카페에서 모던 프렌치 점심  
- 오후: 센 강변 산책 및 부케리 대시장 탐방 (주말이면 열리지 않으니 확인 필요)  
  - 작은 책방과 아티스트 상점 구경  
- 저녁: 센 강변에서 **바토 무슈(유람선 디너)** 예약하여 야경과 함께 식사 (영어 서비스)

---
### 3일차: 몽마르트 언덕 & 현지 예술가 경험
- 오전: **몽마르트르 언덕** 방문  
 

## 8. AutoGen Studio 실행 가이드

AutoGen Studio는 웹 기반의 시각적 인터페이스로, 다양한 에이전트 시나리오를 손쉽게 실험하고 관리할 수 있습니다.

- 설치: 터미널에서 `pip install autogenstudio` 실행
- 실행: 터미널에서 `autogenstudio ui --port 8081` 명령어 입력
- GitHub Codespace가 안내하는 대로 웹 브라우저에서 http://127.0.0.1:8081 으로 접속
- 다양한 프롬프트, 에이전트, 팀 시나리오를 GUI로 실험 가능

### 주요 컴포넌트
- **Agent(에이전트)**: 다양한 역할(예: 플래너, 추천, 요약 등)을 수행하는 AI 에이전트로, 각 에이전트의 시스템 메시지, 프롬프트, 모델 설정 등을 시각적으로 관리할 수 있습니다.
- **Team(팀/그룹)**: 여러 에이전트를 조합해 협업 구조(예: RoundRobin, Selector 등)를 설계하고, 그룹 내 메시지 전달 방식과 종료 조건을 설정할 수 있습니다.
- **Tool(도구)**: 외부 API 호출, 함수 실행 등 에이전트가 활용할 수 있는 다양한 도구를 연결해 복잡한 시나리오를 구현할 수 있습니다.
- **Termination Condition(종료 조건)**: 대화 종료 조건(예: 최대 메시지 수, 특정 키워드 등)을 시각적으로 설정할 수 있습니다.
- **Prompt(프롬프트)**: 각 에이전트 및 팀의 프롬프트를 GUI에서 직접 작성·수정하며, 다양한 프롬프트 엔지니어링 기법을 실험할 수 있습니다.
- **Conversation(대화/실행 결과)**: 에이전트 간 실제 대화 흐름과 결과를 실시간으로 확인하고, 다양한 시나리오를 반복 실험할 수 있습니다.

> AutoGen Studio를 활용하면 복잡한 멀티 에이전트 시스템을 코드 작성 없이 시각적으로 설계·실험할 수 있어, 빠른 프로토타이핑과 실전 적용에 매우 유용합니다.

### AutoGen Studio 관련 가이드 문서

AutoGen Studio의 실제 사용 예시와 다양한 기능, UI 화면은 공식 가이드 문서에서 확인할 수 있습니다.

- [AutoGen Studio 공식 문서](https://microsoft.github.io/autogen/stable/user-guide/autogenstudio-user-guide/index.html)
- [AutoGen Studio GitHub 저장소](https://github.com/microsoft/autogen/tree/main/python/packages/autogen-studio)

위 문서에서는 에이전트, 팀, 도구, 종료 조건 등을 시각적으로 구성하고 실험하는 방법과 다양한 활용 사례를 자세히 안내합니다.

### AutoGen Studio 주요 화면 예시

- ![AutoGen Studio Team Builder](https://github.com/user-attachments/assets/ee5019a1-c27d-48e9-babc-e656da4d7979)
- ![AutoGen Studio Gallery](https://github.com/user-attachments/assets/390d5072-e9ae-4bf5-8d9d-10d2163db7c4)
- ![AutoGen Studio Docker Deploy](https://github.com/user-attachments/assets/f7019d48-9028-4fc4-8e8e-0ec62ec0eaac)

## 9. 마무리 및 다음 단계

이 세션에서는 AutoGen을 활용한 여행 플래너 Multi-Agent 시스템을 구현했습니다. 다양한 역할의 에이전트가 협업하여 사용자의 요구에 맞는 여행 일정을 자동으로 생성하는 과정을 실습했습니다.

- 다양한 역할의 에이전트 설계 및 협업 구조 이해
- RoundRobinGroupChat, SelectorGroupChat 등 다양한 그룹 채팅 옵션 실습
- MaxMessageTermination, TextMentionTermination 등 종료 조건의 실제 적용 예시 실습
- AutoGen Studio를 통한 시각적 에이전트/팀/도구/종료조건 구성 및 실험 방법 학습
- 실전 여행 플래너 예제 구현 및 다양한 시나리오 실험

더 많은 예제와 심화 내용은 아래 자료를 참고하세요.

### 추가 학습 자료
- [AutoGen 공식 문서](https://microsoft.github.io/autogen/)
- [AutoGen Studio 공식 가이드](https://microsoft.github.io/autogen/stable/user-guide/autogenstudio-user-guide/index.html)
- [Generative AI for Beginners (Microsoft)](https://github.com/microsoft/generative-ai-for-beginners): 생성형 AI의 기본 개념과 실습을 단계별로 학습할 수 있는 Microsoft의 오픈 커리큘럼입니다.
- [AI Agents for Beginners (Microsoft)](https://github.com/microsoft/ai-agents-for-beginners): 다양한 AI 에이전트의 원리와 실습을 초보자도 쉽게 따라할 수 있도록 구성한 Microsoft의 공식 자료입니다.